# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets.

## Introduction
Sentiment analysis is to systematically identify, extract, quantify, and study affective states and subjective information based on texts (reference). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the positive or negative (sometimes neutral too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a sentiment analysis package. This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

from nltk.sentiment.vader import SentimentIntensityAnalyzer
txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The tweets data we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis
### Loading and Exploring Data
The dataset we'll be using today is located in the lab directory named Sentiment140.csv.zip. You need to unzip it into a .csv file. Then in the cell below, load and explore the data.

Notes:

The dataset was downloaded from Kaggle. We made a slight change on the original data so that each column has a label.

The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [3]:
import pandas as pd

sentiments = pd.read_csv('../Sentiment140.csv')
sentiments.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [25]:
extract_sentiments = sentiments[:1000]

### Prepare Textual Data for Sentiment Analysis
Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

- clean_up()

- tokenize()

- stem_and_lemmatize()

- remove_stopwords()

Create a new column called text_processed in the dataframe to contain the processed data. At the end, your text_processed column should contain lists of word tokens that are cleaned up. Your data should look like below:

In [16]:
from stop_words import get_stop_words

def remove_stopwords(l, language):
    stop_words = get_stop_words(language)
    new_l = []
    for word in l:
        if word not in stop_words:
            new_l.append(word)
    return(new_l)

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

def stem_and_lemmatize(l):
    new_l = []
    lemmatizer = WordNetLemmatizer()

    for word in l:
        new_l.append(lemmatizer.lemmatize(word))
    return(new_l)

def tokenize(s):
    return(s.lower().split(' '))

import re


def clean_up(s):
    list_of_good_words = []
    if len(s)>0:
        #words_list = s.split(' ')
        words_list = re.split(" |, |'",s)
        for word in words_list:
            if not word.startswith('http'):
                list_of_good_words.append(''.join(e for e in word if e.isalnum() and not e.isdigit()))
    while("" in list_of_good_words) : 
        list_of_good_words.remove("")
    new_string = ' '.join(new_word for new_word in list_of_good_words)
    return(new_string)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/orianegossiaux/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
string_sample = sentiments.iloc[0].text
string_sample

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [33]:
clean_up(string_sample)

'switchfoot Awww that s a bummer You shoulda got David Carr of Third Day to do it D'

In [34]:
tokenize(clean_up(string_sample))

['switchfoot',
 'awww',
 'that',
 's',
 'a',
 'bummer',
 'you',
 'shoulda',
 'got',
 'david',
 'carr',
 'of',
 'third',
 'day',
 'to',
 'do',
 'it',
 'd']

In [35]:
stem_and_lemmatize(tokenize(clean_up(string_sample)))

['switchfoot',
 'awww',
 'that',
 's',
 'a',
 'bummer',
 'you',
 'shoulda',
 'got',
 'david',
 'carr',
 'of',
 'third',
 'day',
 'to',
 'do',
 'it',
 'd']

In [36]:
remove_stopwords(stem_and_lemmatize(tokenize(clean_up(string_sample))), 'en')

['switchfoot',
 'awww',
 's',
 'bummer',
 'shoulda',
 'got',
 'david',
 'carr',
 'third',
 'day',
 'd']

In [26]:
extract_sentiments['text_processed'] = extract_sentiments['text'].map(lambda x : remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x))), 'en'))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
extract_sentiments.head()

,target,id,date,flag,user,text,text_processed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[switchfoot, awww, s, bummer, shoulda, got, da..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, can, t, update, facebook, texting, mig..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dived, many, time, ball, managed, s..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, s, behaving, m, mad, can, t,..."


### Creating Bag of Words
The purpose of this step is to create a bag of words from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. corpus) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in text_processed and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's FreqDist class (documentation). Then select the top 5,000 words from the frequency distribution.

In [52]:
import numpy as np

bags_of_words = {}
for index, row in extract_sentiments.iterrows():
    for word in row['text_processed']:
        if word not in bags_of_words:
            bags_of_words[word] = 1
        else:
            bags_of_words[word] += 1
        
sorted(bags_of_words)


['aaaaand',
 'aaronrva',
 'aaw',
 'able',
 'absolutely',
 'abt',
 'accent',
 'accept',
 'access',
 'accessed',
 'accessing',
 'accident',
 'accidentally',
 'accompaniers',
 'according',
 'account',
 'ache',
 'acronym',
 'acting',
 'activated',
 'actually',
 'ad',
 'adame',
 'adamgoldston',
 'add',
 'added',
 'adidas',
 'admitted',
 'admitting',
 'adopted',
 'advance',
 'affected',
 'afford',
 'affordable',
 'afraid',
 'afternoon',
 'againand',
 'age',
 'agency',
 'aghsnow',
 'ago',
 'agree',
 'agreed',
 'ah',
 'ahead',
 'ahh',
 'ahhhh',
 'ahleyf',
 'aim',
 'aint',
 'airport',
 'akianz',
 'alarm',
 'album',
 'alcohol',
 'alejandralei',
 'alert',
 'alexbigman',
 'alicayaba',
 'alicia',
 'alielayus',
 'alive',
 'allergic',
 'allergy',
 'alliana',
 'allllll',
 'allnighter',
 'allows',
 'allyheman',
 'almost',
 'alone',
 'along',
 'alot',
 'already',
 'also',
 'although',
 'altrincham',
 'always',
 'alydesigns',
 'amandaenglund',
 'amanorris',
 'amazing',
 'amazon',
 'amberbenson',
 'amberp